# Raw data
--------------
## first check the raw data

In [4]:
# check how many lines
! wc -l data/emojitweets-01-04-2018.txt

18866900 data/emojitweets-01-04-2018.txt


In [2]:
# check how many words
! wc -w data/emojitweets-01-04-2018.txt

209430000 data/emojitweets-01-04-2018.txt


In [3]:
# On average the sentense are have 11 words
209430000/18866900

11.100392751326398

## Train - test - validation split

In [6]:
# split on 80-10-10
18866900 * 0.8, 18866900 * 0.1, 18866900 * 0.2

(15093520.0, 1886690.0, 3773380.0)

In [8]:
! head -15093520 data/emojitweets-01-04-2018.txt >>emoji_train.txt


In [14]:
! (tail -3773380 data/emojitweets-01-04-2018.txt | head -1886690) >> emoji_val.txt

tail: error writing 'standard output': Broken pipe


In [10]:
! tail -1886690 data/emojitweets-01-04-2018.txt >> emoji_test.txt

In [51]:
# generate small_sample
! head -1509352 emoji_train.txt >>emoji_ss.txt

# Word Vectorize

In [1]:
# Python program to generate word vectors using Word2Vec 
  
# importing all necessary modules 
from nltk.tokenize import sent_tokenize, word_tokenize 
import warnings 
  
warnings.filterwarnings(action = 'ignore') 
  
import gensim 
from gensim.models import Word2Vec 
  
# sample = open("emoji_1M.txt", "r") 
sample = open("emoji_train.txt", "r") 
s = sample.read() 
  
# Replaces escape character with space 
f = s.replace("\n", " ") 
  
data = [] 
  

In [3]:
import emoji
import regex

def split_count(text):

    emoji_list = []
    data = regex.findall(r'\X', text)
    for word in data:
        if any(char in emoji.UNICODE_EMOJI for char in word):
            emoji_list.append(word)

    return emoji_list

In [54]:
# 
sample = open("emoji_ss.txt", "r") 
s_ss = sample.read() 
  
# Replaces escape character with space 
f_ss = s_ss.replace("\n", " ") 
  
data = [] 

In [ ]:
s_ss

In [ ]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import pos_tag
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

# Tokenize and stop words

# 1. Create a set of documents.
documents = [' '.join(article['content']).lower() for article in coll.find()]

# 2. Create a set of tokenized documents.
docs = [word_tokenize(content) for content in documents]

# 3. Strip out stop words from each tokenized document.
stop = set(stopwords.words('english'))
docs = [[word for word in words if word not in stop] for words in docs]

In [4]:
emoji_set = set(split_count(f))

In [12]:
# iterate through each sentence in the file 
for i in sent_tokenize(f): 
    temp = [] 
      
    # tokenize the sentence into words 
    for j in word_tokenize(i): 
        temp.append(j.lower()) 
  
    data.append(temp) 

In [10]:
emoji_list = list(emoji_set)
with open("emoji_list.csv", 'w', newline='') as myfile:
     w = csv.writer(myfile, quoting=csv.QUOTE_ALL)
     w.writerow(emoji_list)

4358

In [52]:
data[0]

['squad',
 'arriving',
 'for',
 'game',
 '2',
 '🚀',
 'dude',
 'is',
 'like',
 '5',
 '’',
 '8',
 '140',
 'pounds',
 'his',
 'dick',
 'was',
 'long',
 'and',
 'strong',
 '(',
 'always',
 'the',
 'little',
 'dudes',
 'carrying',
 'the',
 '🍆',
 ')',
 '\U0001f92a🙃',
 'followers👇',
 'i',
 'cant',
 'breatiuhw',
 '💀💀💀',
 '2️⃣4️⃣',
 'hours',
 "'til",
 'our',
 'schedule',
 'drops',
 '!']

In [13]:
# Create CBOW model 
model1 = gensim.models.Word2Vec(data, min_count = 1,  
                              size = 100, window = 5) 

In [15]:
# Save CBOW model
import pickle
with open('cbow.pkl', 'wb') as f:
    pickle.dump(model1, f)

In [18]:
model1.estimate_memory()

{'vocab': 1606239500,
 'vectors': 1284991600,
 'syn1neg': 1284991600,
 'total': 4176222700}

In [22]:
word = "bike"
sim = model1.most_similar(word,topn=50)
for i in sim:
    if i in emoji_list:
        return i

[('motorcycle', 0.7577804327011108),
 ('yacht', 0.6977012157440186),
 ('🚲', 0.6723682880401611),
 ('bicycle', 0.6688296794891357),
 ('bikes', 0.6612945199012756),
 ('horse', 0.6549836993217468),
 ('mattress', 0.6493205428123474),
 ('buggy', 0.6399540901184082),
 ('car', 0.6371113061904907),
 ('engine', 0.6342647075653076),
 ('lawn', 0.6337512135505676),
 ('truck', 0.6288256645202637),
 ('kayak', 0.6270779371261597),
 ('motorbike', 0.6254085898399353),
 ('jet', 0.6219853758811951),
 ('ferrari', 0.6187005639076233),
 ('vehicle', 0.6166753768920898),
 ('jeep', 0.6139025688171387),
 ('horseback', 0.6051432490348816),
 ('tractor', 0.6044219732284546),
 ('paddle', 0.603623628616333),
 ('tesla', 0.6030915975570679),
 ('🚘', 0.6017401218414307),
 ('route', 0.5992273688316345),
 ('yeti', 0.5969487428665161),
 ('towel', 0.5940559506416321),
 ('skateboard', 0.5918545126914978),
 ('honda', 0.5908911824226379),
 ('rear', 0.5888210535049438),
 ('highway', 0.5880736112594604),
 ('rug', 0.5873566269874

In [27]:
emoji_list = list(emoji_set)

In [45]:
def emoji_prodictor(word):
    sim = model1.most_similar(word,topn=50)
    for i in sim:
        if i[0] in emoji_list:
            return i[0]
    return word

In [47]:
s1 = "let's go grocery today to get some bike related stuff"
for w in s1.lower().split(" "):
    print(emoji_prodictor(w))


let's
go
grocery
today
🤷🏽‍♀️
get
😂
🚲
related
stuff


In [44]:
emoji_prodictor("bike")

'bike'